<a href="https://colab.research.google.com/github/evanfan2040/python-virtual-stock-trading/blob/main/PythonFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install twstock

In [ ]:
!pip install yfinance

In [ ]:
!pip install pandas

In [ ]:
!pip install plotly

In [ ]:
!pip install numpy

In [ ]:
!pip install shioaji

##**測試區**
這裡是在實作用爬蟲抓到即時的股票大部分的資料

In [ ]:
import twstock as tw
import pandas as pd
num = input('請輸入證券代碼')
stock = tw.realtime.get(num)
#print(stock['success'])

result = pd.DataFrame(stock).T.iloc[1:3]
result.columns = ['code', 'location', 'stock_name', 'company_name', 'time', 'latest_sellprice', 'volume', 'overall_volume', 'topfive_sellprice', 'topfive_sellvolume', 'topfive_buyprice', 'topfive_buyvolume', 'opening_price', 'highestprice', 'lowestprice']
result

這裡是在實作用爬蟲抓到即時的股票價格

In [ ]:
from posix import X_OK
from os import PRIO_PROCESS
import requests
from bs4 import BeautifulSoup
code = input('請輸入代碼')

url = 'https://tw.stock.yahoo.com/quote/'+code
web = requests.get(url)
soup = BeautifulSoup(web.text, "html.parser")
X = soup.select('.Fz\(32px\)')[0]
print(X.get_text())
price = f'{X.get_text()}'
price = price.replace(',', '')

price = int(price)
print(price)

實作繼承帳號資訊的文件，但由於文件的形式有點不方便，就使用json來做為資料的存放處

In [ ]:
FileName = input('請輸入學號')
file = open(FileName, mode = 'w')
file.write('hello')
file.close()

## **導入區**
這裡可以將預設檔案導入左邊的檔案區，這樣這個專案的程式都讀取的到，會需要三個檔案，分別是config.json、stock.txt和trade.txt

In [3]:
from google.colab import files
uploaded = files.upload()

Saving config.json to config.json


## **手動重製區**
這裡是手動的調整區，可以調整這個帳號的基礎資料，像是目前持有金錢


In [12]:
import json
import os
data["money"] = 100000
data["active"] = "no"
data["name"] = ""
with open("config.json", mode = 'w') as file:
  json.dump(data, file)
with open("stock.txt", mode = 'w') as file:
   file.write("")
with open("trade.txt", mode = 'w') as file:
   file.write("")
print('已重製')

已重製


# **下方的程式是主區域，按下啟動之後可以做以下的六個動作**

*   買入(輸入1):根據現在市場的價格，購入輸入數量的股票

---

*   賣出(輸入2):根據現在市場的價格，賣出輸入數量的股票

---

*   查詢個人資訊(輸入3):查詢餘額，擁有的股票股數

---

*   查詢股票資訊(輸入4):查詢當下股票的市場價

---


*   重製(輸入redo):重製整支帳戶的基礎資訊，包括帳戶名稱跟起始基金

---


*   結束(輸入0):結束此程序





# **程式講解:**
首先導入需要的api，我們分別需要

1.  json:管理及儲存帳戶
2.  os:去管理帳號及交易資訊
3.  requests:讀取網站內容
4.  bs4:這邊使用了BeautifulSoup去轉譯網站的資訊



```
import json
import os
import requests
from bs4 import BeautifulSoup
```
第一步是基礎的帳號架設，會使用到json去储存帳號的基礎資訊，這裡使用的json檔案已經有初步的設定了，輸入學號後，就正式開通帳號

> *如果還未導入json，可以使用預設在檔案的config.json加上上面的導入區做導入*



```
# with open("config.json", mode = 'r') as file:
  data = json.load(file)
if data["name"] == "":
  StudentID = input('請輸入學號')
active = data['active']
money = data['money']
name = data['name']
if active == "no":
  data["name"] = StudentID
  data["active"] = "yes"
```



---



這段是代表股票價格導入的函式，當他接受到code後會利用requests到對應的網站擷取股票的價格，在導入之後，經由BeautifulSoup轉譯內容，擷取在Fz(32)這個class下的文字，也就是市場價，最後回傳至呼叫的程式
```
# def load(code):
    url = 'https://tw.stock.yahoo.com/quote/'+code
    web = requests.get(url)
    soup = BeautifulSoup(web.text, "html.parser")
    X = soup.select('.Fz\(32px\)')[0]
    price = f'{X.get_text()}'
    price = price.replace(',', '')
    return price
```



---


第二步就是執行股票交易相關的動作


*  買入(輸入1)


系統會要求輸入股票代碼，接著系統會根據輸入的代碼，發送load(code)呼叫函式，擷取價格，假如價格為"-"，表示此股票不存在，輸入要購買的數量後，假設錢包的錢不夠，跳出餘額不足，其中由於股票代碼為英文的都使用美金，所以暫定用price*30表示匯率




```
# if move == '1':
    code = input('請輸入股票代碼')
    price = load(code)
    if price == '-':
      print('無此股票')
      continue
    price = float(price)
    if code.isalpha() == True:
      price = price * 30
    share = float(input('請輸入數量'))
    if money < price * share:
      print('餘額不足')
      continue
```
接下來的這段是模擬錢包跟交易紀錄的代碼，利用os中的寫入跟讀取去計算錢包中的金額，並且寫入購入的股票的代碼、數量、以及當下平均的總金額(股票平均價格*股票數量)，當下的資產狀況會寫入stock.txt，交易紀錄則會寫入trade.txt



```
#   with open("trade.txt", mode = 'a') as file:
      file.write(code+', '+str(share)+', 買入\n')
    price = price * share
    money = money - price
    data["money"] = money

    with open("stock.txt", mode = 'r') as file:
      lines = file.readlines()
      #寫入的方式為全讀取後找到對應的行列，在修改後重新寫入
    with open("stock.txt", mode = 'r+') as file:
      exist = False
      for i in range(len(lines)):
        info = lines[i].split()
        if info[0] == code:
            info[1] = float(info[1])
            info[2] = float(info[2])
            info[1] = info[1] + share
            info[2] = info[2] + price
            lines[i] = info[0] + " " + str(info[1]) + " " + str(info[2]) + "\n"
            exist = True
            
      with open("stock.txt", mode='w') as file:
          file.writelines(lines)
      if exist == False:  #未存在的行列會新增在最底下
        with open("stock.txt", mode='a') as file:
                file.write(str(code) + " " + str(share) + " " + str(price) + "\n")
        with open("config.json", mode='w') as file:
            json.dump(data, file)
```

*  賣出(輸入2)

賣出的邏輯跟買入一致，偵測到股票數量不夠會跳出執行，而偵測到股票都賣完則會將整行的資料清空
```
#   #這裡標示出跟買入不同的部分  
    with open("stock.txt", mode = 'r+') as file:
      for i in range(len(lines)):
        info = lines[i].split()
        if info[0] == code:
            info[1] = float(info[1])
            info[2] = float(info[2])
            avg = info[2] / info[1]
            info[1] = info[1] - share
            info[2] = info[2] - avg * share
            if info[1] == 0:
              lines.pop(i)
            elif info[1] < 0:
              print('數量不足')
              continue
            else:
              lines[i] = info[0] + " " + str(info[1]) + " " + str(info[2]) + "\n"
```

*  查詢個人資訊(輸入3)

利用os讀取stock.txt中的持股資料以及config.json中的money常數，並顯示給使用者看



```
# print(f'目前餘額為{money}新台幣\n')
    with open("stock.txt", mode = 'r') as file:
      for line in file:
        info = line.split()
        value = float(info[2])
        print(f'股票代碼{info[0]} 股票數量{info[1]}股 股票平均總價值%.2f新台幣\n' %(value))
```

*  查詢公司股票資訊(輸入4)

利用load()函數讀取股票市場價並展示給使用者



```
# code = input('請輸入股票代碼')
  price = load(code)
  if price == '-':
    print('無此股票')
    continue
  price = float(price)
  if code.isalpha() == True:
    price = price * 30  
  print(f"目前價格為{price}新台幣\n")
```

*  重製(輸入redo)

清空三個檔案的資訊，並強制結束程式



```
#   data["money"] = 100000
    data["active"] = "no"
    data["name"] = ""
    with open("config.json", mode = 'w') as file:
      json.dump(data, file)
    with open("stock.txt", mode = 'w') as file:
      file.write("")
    with open("trade.txt", mode = 'w') as file:
      file.write("")
    print('已重製')
    break
```
以上是這個專案的全部




In [11]:
import json
import os
import requests
from bs4 import BeautifulSoup
def load(code):
    url = 'https://tw.stock.yahoo.com/quote/'+code
    web = requests.get(url)
    soup = BeautifulSoup(web.text, "html.parser")
    X = soup.select('.Fz\(32px\)')[0]
    price = f'{X.get_text()}'
    price = price.replace(',', '')
    return price

with open("config.json", mode = 'r') as file:
  data = json.load(file)
if data["name"] == "":
  StudentID = input('請輸入學號')
active = data['active']
money = data['money']
name = data['name']
if active == "no":
  data["name"] = StudentID
  data["active"] = "yes"

while(True):
  move = input('買入請輸入1，賣出請輸入2, 查詢個人資料請輸入3, 查詢公司股價請輸入4, 重製請輸入redo, 關閉請輸入0')

  if move == '1':
    code = input('請輸入股票代碼')
    price = load(code)
    if price == '-':
      print('無此股票')
      continue
    price = float(price)
    if code.isalpha() == True:
      price = price * 30
    share = float(input('請輸入數量'))
    if money < price * share:
      print('餘額不足')
      continue
    with open("trade.txt", mode = 'a') as file:
      file.write(code+', '+str(share)+', 買入\n')
    price = price * share
    money = money - price
    data["money"] = money

    with open("stock.txt", mode = 'r') as file:
      lines = file.readlines()

    with open("stock.txt", mode = 'r+') as file:
      exist = False
      for i in range(len(lines)):
        info = lines[i].split()
        if info[0] == code:
            info[1] = float(info[1])
            info[2] = float(info[2])
            info[1] = info[1] + share
            info[2] = info[2] + price
            lines[i] = info[0] + " " + str(info[1]) + " " + str(info[2]) + "\n"
            exist = True

      with open("stock.txt", mode='w') as file:
          file.writelines(lines)
      if exist == False:
        with open("stock.txt", mode='a') as file:
                file.write(str(code) + " " + str(share) + " " + str(price) + "\n")
        with open("config.json", mode='w') as file:
            json.dump(data, file)

  elif move == '2':
    code = input('請輸入股票代碼')
    price = load(code)
    if price == '-':
      print('無此股票')
      continue
    price = float(price)
    if code.isalpha() == True:
      price = price * 30
    share = float(input('請輸入數量'))

    with open("trade.txt", mode = 'a') as file:
      file.write(code+', '+str(share)+', 賣出\n')
    price = price * share
    money = money + price
    data["money"] = money

    with open("stock.txt", mode = 'r') as file:
      lines = file.readlines()

    with open("stock.txt", mode = 'r+') as file:
      for i in range(len(lines)):
        info = lines[i].split()
        if info[0] == code:
            info[1] = float(info[1])
            info[2] = float(info[2])
            avg = info[2] / info[1]
            info[1] = info[1] - share
            info[2] = info[2] - avg * share
            if info[1] == 0:
              lines.pop(i)
            elif info[1] < 0:
              print('數量不足')
              continue
            else:
              lines[i] = info[0] + " " + str(info[1]) + " " + str(info[2]) + "\n"


    with open("stock.txt", mode='w') as file:
          file.writelines(lines)

    with open("config.json", mode='w') as file:
         json.dump(data, file)

  elif move == '3':
    print(f'目前餘額為{money}新台幣\n')
    with open("stock.txt", mode = 'r') as file:
      for line in file:
        info = line.split()
        value = float(info[2])
        print(f'股票代碼{info[0]} 股票數量{info[1]}股 股票平均總價值%.2f新台幣\n' %(value))

  elif move == '4':
    code = input('請輸入股票代碼')
    price = load(code)
    if price == '-':
      print('無此股票')
      continue
    price = float(price)
    if code.isalpha() == True:
      price = price * 30
    print(f"目前價格為{price}新台幣\n")

  elif move == 'redo':
    data["money"] = 100000
    data["active"] = "no"
    data["name"] = ""
    with open("config.json", mode = 'w') as file:
      json.dump(data, file)
    with open("stock.txt", mode = 'w') as file:
      file.write("")
    with open("trade.txt", mode = 'w') as file:
      file.write("")
    print('已重製')
    break

  elif move == '0':
    print('程式已結束')
    break


買入請輸入1，賣出請輸入2, 查詢個人資料請輸入3, 查詢公司股價請輸入4, 重製請輸入redo, 關閉請輸入01
請輸入股票代碼2330
請輸入數量6
買入請輸入1，賣出請輸入2, 查詢個人資料請輸入3, 查詢公司股價請輸入4, 重製請輸入redo, 關閉請輸入03
目前餘額為87570.0新台幣

股票代碼2330 股票數量6.0股 股票平均總價值6780.00新台幣

買入請輸入1，賣出請輸入2, 查詢個人資料請輸入3, 查詢公司股價請輸入4, 重製請輸入redo, 關閉請輸入02
請輸入股票代碼2330
請輸入數量3
買入請輸入1，賣出請輸入2, 查詢個人資料請輸入3, 查詢公司股價請輸入4, 重製請輸入redo, 關閉請輸入03
目前餘額為90960.0新台幣

股票代碼2330 股票數量3.0股 股票平均總價值3390.00新台幣

買入請輸入1，賣出請輸入2, 查詢個人資料請輸入3, 查詢公司股價請輸入4, 重製請輸入redo, 關閉請輸入00
程式已結束


## 這裡是下載區
可以將三個檔案下載下來，防止檔案在colab離線時被丟棄。

In [5]:
from google.colab import files
files.download("stock.txt")
files.download("trade.txt")
files.download("config.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>